In [29]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from pypdf import PdfReader
import zxing
import pandas as pd
%matplotlib inline

In [30]:
pdf_link = "file1.pdf"
reader = PdfReader(pdf_link)

number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [31]:
def process(page_number):
    page = reader.pages[page_number]
    text = page.extract_text()

    texts = text.split('\n')
    col_1 = []
    col_8 = []
    for i in texts:
        if str.isdecimal(i):
            if (len(i) == 9):
                col_1.append(i)
            else:
                col_8.append(i)

    code_1 = []
    code_2 = []
    for item in range(len(col_8)):
        code_1.append(col_8[item][0:6])
        code_2.append(col_8[item][6:])  
    
    return col_1, code_1, code_2

In [32]:
col_1 = []
code_1 = [] 
code_2 = []
for i in range(number_of_pages):
    a, b, c = process(i)
    col_1 = col_1 + a
    code_1 = code_1 + b
    code_2 = code_2 + c


print(len(col_1))
print(len(code_1))
print(len(code_2))

46
46
46


In [33]:
import pypdfium2 as pdfium

# Load a document
pdf = pdfium.PdfDocument(pdf_link)

# Loop over pages and render
for i in range(len(pdf)):
    page = pdf[i]
    image = page.render(scale=4).to_pil()
    image.save('pdf2jpg1/page'+ str(i) +'.jpg', 'JPEG')

In [34]:
def blur_image(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, img = cv.threshold(img, 100, 255, cv.THRESH_BINARY)
    img = cv.blur(img, (10,10))
    ret, img = cv.threshold(img, 230, 255, cv.THRESH_BINARY)
    img = cv.blur(img, (5,5))
    ret, img = cv.threshold(img, 230, 255, cv.THRESH_BINARY)
    return img

# a = blur_image(img)
# plt.imshow(a, cmap='gray')

In [35]:
def get_rectangle_position(img):
    # Đọc ảnh gray scale
    # img = cv.imread('th.jpg', cv.IMREAD_GRAYSCALE)
    img = blur_image(img)

    # Áp dụng ngưỡng nhị phân để chuyển ảnh thành ảnh nhị phân
    _, thresh = cv.threshold(img, 127, 255, cv.THRESH_BINARY)

    # Tìm các đường viền trong ảnh nhị phân
    contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Lọc ra các hình chữ nhật màu đen
    rectangles = []
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        if w > 50 and h > 50 and w < 0.9 * img.shape[1] and h < 0.9 * img.shape[0]:
            roi = img[y:y+h, x:x+w]
            if cv.mean(roi)[0] < 50:
                rectangles.append((x, y, w, h))
    
    # # Vẽ các hình chữ nhật lên ảnh gốc
    # img = cv.imread('th.jpg')
    # for (x, y, w, h) in rectangles:
    #     cv.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    # plt.imshow(img)
    # cv.imwrite('th1.jpg', img)
    rectangles.sort()
    return rectangles

In [40]:
reader = zxing.BarCodeReader()
barcode_detect_list = []
for i in range(number_of_pages):
    img = cv.imread("pdf2jpg/page"+str(i)+".jpg")

    rectangles = get_rectangle_position(img)
    # print(rectangles)
    for (x, y, w, h) in rectangles:
        rs = ''
        barcode = img[y:y+h, x:x+w,:]
        ret, barcode = cv.threshold(barcode, 127, 255, cv.THRESH_BINARY)

        cv.imwrite('cuongpc.jpg', barcode)

        rs = reader.decode('cuongpc.jpg').raw   
    
        barcode_detect_list.append(rs)
        print(rs)
        # if len(detectedBarcodes) == 0:
        #     barcode_detect_list.append('Fail')   
        # else:
        #     barcode_data = detectedBarcodes[0]
        #     barcode_detect_list.append(barcode_data.data)

    # cv.imwrite("pdf2jpg_out/page"+str(i)+".jpg", img)

*bZm44Rz
*bZqE/iE
*bZjuJ4C
*bZZbvJh
*bZnhtKZ
*bZSoQYG
*bZTgYoA
*bZmYGh4
*bZlfNkQ
*bZP2vJz
*bZQ5GSX
*bZXL+CX
*bZKAXMj
*bZl5f8T
*bZl5f4S
*bZUVZ6I
*bZqcgKE
*bZUVZ2J
*bZQr6OZ
*bZZYfpn
*bZKs643
*bZKs68x
*bZeoV8V
*bZUEUGE
*bZJ9D0l
*bZf28NC
*bZeCGV5
*bZSuo0D
*bZbvKwi
*bZKkdcy
*bZVHpWC
*bZn7UyR
*bZaivdS
*bZbUick
*bZKbokg
*bZJPMgj
*bZWRU02
*bZNp9MQ
*bZNN09m
*bZMCtVh
*bZn9XWZ
*bZI86om
*bZYTA1m
*bZSuowA
*bZc7+h2
*bZM3lBm


In [37]:
data = {
  "col_1": col_1,
  "code_1": code_1,
  "code_2": code_2,
  "barcode_detect_list": barcode_detect_list,

}
df = pd.DataFrame(data)
df.to_excel('output.xlsx')